In [2]:
import pandas as pd
import gensim
from gensim.models import KeyedVectors
from stellargraph.data import EdgeSplitter
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV

2023-02-21 21:40:21.099096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 21:40:21.253270: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-21 21:40:21.257711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 21:40:21.257731: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [65]:
model = gensim.models.Word2Vec.load('../lm-vol/2_10_word2vec.model')
wv = KeyedVectors.load('../lm-vol/2_10_word2vec.wordvectors', mmap='r')
G = nx.read_graphml('../lm-vol/LANL_train.graphml')
def operator_hadamard(u, v):
    return u * v
node_embeddings = (wv.vectors) 
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, edges_train, labels_train = edge_splitter_test.train_test_split(
    p=0.99, method="global"
)
training_edges = [operator_hadamard(wv[edge[0]], wv[edge[1]]) for edge in edges_train]
clf = LogisticRegressionCV(cv=5, max_iter=1000, random_state=0).fit(training_edges, labels_train)
clf.score(training_edges, labels_train)

** Sampled 630767 positive and 630767 negative edges. **


0.9828629271981572

In [ ]:
graph_structure = (1, 3, 2)
inconclusive = 0
n = 0
with open('../lm-vol/LANL_anomalies.csv', 'a') as f:
    for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
    #     display(chunk)
        chunk_invalids = pd.DataFrame()
        client_to_ip_pred = []
        ip_to_service_pred = []
        for index, row in chunk.iterrows():
            try: 
                client_embedding = wv[(row[graph_structure[0]])]
                ip_embedding = wv[(row[graph_structure[1]])]
                service_embedding = wv[(row[graph_structure[2]])]
                client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
                ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
                client_to_ip = clf.predict_proba(client_to_ip_embedding)
                ip_to_service = clf.predict_proba(ip_to_service_embedding)
                client_to_ip_pred.append(client_to_ip[0][1])
                ip_to_service_pred.append(ip_to_service[0][1])
            except: 
                client_to_ip_pred.append(None)
                ip_to_service_pred.append(None)
                inconclusive += 1
        chunk['client_to_ip_pred'] = client_to_ip_pred
        chunk['ip_to_service_pred'] = ip_to_service_pred
        chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
        chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
        display(chunk_anomalies)
        chunk_anomalies.to_csv(f)
        f.write('\n')

,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,1,2.836545e-19,2.836545e-19
247,2,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C3896,C586,NTLM,Network,LogOn,Success,1,5.435832e-02,5.435832e-02
266,2,C1073$@DOM1,SYSTEM@C1073,C1073,C1073,Negotiate,Service,LogOn,Success,1,6.905511e-02,9.977471e-01
359,2,C1450$@DOM1,SYSTEM@C1450,C1450,C1450,Negotiate,Service,LogOn,Success,1,1.611128e-02,9.685317e-01
417,2,C1976$@DOM1,SYSTEM@C1976,C1976,C1976,Negotiate,Service,LogOn,Success,1,8.053970e-02,9.129982e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
999809,10207,C123$@DOM1,C123$@DOM1,C527,C625,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
999847,10207,C3756$@DOM1,C3756$@DOM1,C3756,TGT,?,?,TGS,Success,1,2.990570e-02,2.990570e-02
999859,10207,C4241$@DOM1,C4241$@DOM1,C4241,C528,Kerberos,Network,LogOn,Success,1,5.771005e-02,5.771005e-02
999889,10207,U281@DOM1,U281@DOM1,C1063,C625,Kerberos,Network,LogOn,Success,1,3.965483e-02,3.965483e-02


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
1000019,10209,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C2719,C586,NTLM,Network,LogOn,Success,1,2.606757e-02,2.606757e-02
1000020,10209,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,1,2.836545e-19,2.836545e-19
1000021,10209,ANONYMOUS LOGON@C625,ANONYMOUS LOGON@C625,C625,C625,?,Network,LogOff,Success,1,2.821142e-16,2.821142e-16
1000033,10209,C123$@DOM1,C123$@DOM1,C527,C1085,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
1000123,10210,C123$@DOM1,C123$@DOM1,C527,C1065,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
1999878,20375,C123$@DOM1,C123$@DOM1,C527,C101,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
1999879,20375,C123$@DOM1,C123$@DOM1,C527,C529,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
1999942,20375,U1733@DOM1,U1733@DOM1,C5850,C2327,?,?,TGS,Success,1,7.706725e-02,7.706725e-02
1999943,20375,U1733@DOM1,U1733@DOM1,C5850,C5850,?,?,TGT,Success,1,7.706725e-02,7.706725e-02


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
2000045,20376,U1733@DOM1,U1733@DOM1,C5850,C2327,?,?,TGS,Success,1,7.706725e-02,7.706725e-02
2000046,20376,U1733@DOM1,U1733@DOM1,C5850,C5850,?,?,TGT,Success,1,7.706725e-02,7.706725e-02
2000066,20377,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,1,2.836545e-19,2.836545e-19
2000073,20377,C123$@DOM1,C123$@DOM1,C527,C529,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
2000074,20377,C123$@DOM1,C123$@DOM1,C527,C553,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
2999887,26606,U1685@DOM1,U1685@DOM1,C529,C529,?,Network,LogOff,Success,1,2.749932e-02,2.749932e-02
2999901,26606,U3091@DOM1,U3091@DOM1,C3750,C1065,?,?,TGS,Success,1,5.647916e-02,5.647916e-02
2999956,26607,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,1,8.572672e-19,8.572672e-19
2999958,26607,ANONYMOUS LOGON@C754,ANONYMOUS LOGON@C754,C754,C754,?,Network,LogOff,Success,1,4.021694e-19,4.021694e-19


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
3000068,26607,C8137$@DOM1,C8137$@DOM1,C8137,C231,Kerberos,Network,LogOn,Success,1,8.241153e-02,8.241153e-02
3000069,26607,C8137$@DOM1,C8137$@DOM1,C8137,C706,Kerberos,Network,LogOn,Success,1,8.241153e-02,8.241153e-02
3000077,26607,C8557$@DOM1,C8557$@DOM1,C8557,C625,Kerberos,Network,LogOn,Success,1,5.560019e-02,5.560019e-02
3000082,26607,C8613$@DOM1,C8613$@DOM1,C8613,C231,?,?,TGS,Success,1,5.149835e-02,5.149835e-02
3000083,26607,C8613$@DOM1,C8613$@DOM1,C8613,C231,Kerberos,Network,LogOn,Success,1,5.149835e-02,5.149835e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
3999889,30320,ANONYMOUS LOGON@C612,ANONYMOUS LOGON@C612,C612,C612,?,Network,LogOff,Success,1,3.578713e-17,3.578713e-17
3999893,30320,ANONYMOUS LOGON@C754,ANONYMOUS LOGON@C754,C754,C754,?,Network,LogOff,Success,1,4.021694e-19,4.021694e-19
3999923,30320,C11656$@DOM1,C11656$@DOM1,C231,C231,?,Network,LogOff,Success,1,2.235674e-02,2.235674e-02
3999938,30320,C123$@DOM1,C123$@DOM1,C527,C1085,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
4000117,30320,U1653@DOM1,U1653@DOM1,C754,C754,?,Network,LogOff,Success,1,1.320976e-13,1.320976e-13
4000119,30320,U1718@?,U1718@?,C5787,C6495,NTLM,Network,LogOn,Fail,1,4.100460e-03,4.100460e-03
4000218,30321,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,1,2.836545e-19,2.836545e-19
4000253,30321,C123$@DOM1,C123$@DOM1,C527,C523,Kerberos,Network,LogOn,Success,1,3.111875e-02,3.111875e-02
4000444,30322,ANONYMOUS LOGON@C2106,ANONYMOUS LOGON@C2106,C2106,C2106,?,Network,LogOff,Success,1,3.790263e-20,3.790263e-20
...,...,...,...,...,...,...,...,...,...,...,...,...
4999650,34009,C5194$@DOM1,C5194$@DOM1,C5194,C528,Kerberos,Network,LogOn,Success,1,9.137759e-02,9.137759e-02
4999651,34009,C5194$@DOM1,C5194$@DOM1,C5194,TGT,?,?,TGS,Success,1,9.137759e-02,9.137759e-02
4999810,34009,U838@DOM1,U838@DOM1,C4525,C467,Kerberos,Network,LogOn,Success,1,8.675935e-02,8.675935e-02
4999821,34010,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C12185,C586,NTLM,Network,LogOn,Success,1,4.816970e-02,4.816970e-02


...

,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
261000038,2239008,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,26,2.836545e-19,2.836545e-19
261000040,2239008,ANONYMOUS LOGON@C612,ANONYMOUS LOGON@C612,C612,C612,?,Network,LogOff,Success,26,3.578713e-17,3.578713e-17
261000086,2239008,C19752$@DOM1,C19752$@DOM1,C19752,C19752,?,Network,LogOff,Success,26,2.733972e-02,2.733972e-02
261000087,2239008,C19752$@DOM1,C19752$@DOM1,C19752,C612,Kerberos,Network,LogOn,Success,26,2.733972e-02,2.733972e-02
261000088,2239008,C19752$@DOM1,C19752$@DOM1,C19752,C625,Kerberos,Network,LogOn,Success,26,2.733972e-02,2.733972e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
261999856,2245492,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,26,2.836545e-19,2.836545e-19
261999983,2245493,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,26,1.883311e-17,1.883311e-17
261999984,2245493,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,26,2.836545e-19,2.836545e-19
261999990,2245493,C13525$@DOM1,C13525$@DOM1,C13525,C625,Kerberos,Network,LogOn,Success,26,5.148778e-02,5.148778e-02


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
262000099,2245494,ANONYMOUS LOGON@C1065,ANONYMOUS LOGON@C1065,C1065,C1065,?,Network,LogOff,Success,26,4.464676e-18,4.464676e-18
262000102,2245494,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,26,2.836545e-19,2.836545e-19
262000117,2245494,C123$@DOM1,C123$@DOM1,C527,C988,Kerberos,Network,LogOn,Success,26,3.111875e-02,3.111875e-02
262000244,2245494,U6539@DOM1,U6539@DOM1,C2600,C612,Kerberos,Network,LogOn,Success,26,9.632488e-02,9.632488e-02
262000266,2245495,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,26,2.836545e-19,2.836545e-19
...,...,...,...,...,...,...,...,...,...,...,...,...
262999845,2252015,C10321$@DOM1,C10321$@DOM1,C10321,C2106,Kerberos,Network,LogOn,Success,27,2.198800e-02,2.198800e-02
262999865,2252015,C1640$@DOM1,C1640$@DOM1,C2109,C2106,Kerberos,Network,LogOn,Success,27,1.181980e-05,1.181980e-05
262999933,2252015,C7778$@DOM1,C7778$@DOM1,C7778,C2106,Kerberos,Network,LogOn,Success,27,8.923221e-02,8.923221e-02
262999977,2252016,ANONYMOUS LOGON@C1065,ANONYMOUS LOGON@C1065,C1065,C1065,?,Network,LogOff,Success,27,4.464676e-18,4.464676e-18


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
263000115,2252017,ANONYMOUS LOGON@C2106,ANONYMOUS LOGON@C2106,C2106,C2106,?,Network,LogOff,Success,27,3.790263e-20,3.790263e-20
263000119,2252017,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,27,2.836545e-19,2.836545e-19
263000147,2252017,C1640$@DOM1,C1640$@DOM1,C2109,C2106,Kerberos,Network,LogOn,Success,27,1.181980e-05,1.181980e-05
263000243,2252018,ANONYMOUS LOGON@C528,ANONYMOUS LOGON@C528,C528,C528,?,Network,LogOff,Success,27,2.598007e-17,2.598007e-17
263000244,2252018,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,27,2.836545e-19,2.836545e-19
...,...,...,...,...,...,...,...,...,...,...,...,...
263999774,2258602,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,27,1.883311e-17,1.883311e-17
263999777,2258602,ANONYMOUS LOGON@C529,ANONYMOUS LOGON@C529,C529,C529,?,Network,LogOff,Success,27,1.548829e-17,1.548829e-17
263999778,2258602,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,27,2.836545e-19,2.836545e-19
263999783,2258602,C123$@DOM1,C123$@DOM1,C527,C123,?,?,TGS,Success,27,3.111875e-02,3.111875e-02


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
264000060,2258604,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,27,2.836545e-19,2.836545e-19
264000071,2258604,C123$@DOM1,C123$@DOM1,C527,C101,Kerberos,Network,LogOn,Success,27,3.111875e-02,3.111875e-02
264000089,2258604,C1957$@DOM1,C1957$@DOM1,C1957,C586,Kerberos,Network,LogOn,Success,27,7.240992e-02,7.240992e-02
264000114,2258604,C4501$@DOM1,C4501$@DOM1,C4501,C612,Kerberos,Network,LogOn,Success,27,8.444238e-02,8.444238e-02
264000151,2258604,C8908$@DOM1,C8908$@DOM1,C8908,C467,Kerberos,Network,LogOn,Success,27,2.821444e-02,2.821444e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
264999801,2265212,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,27,2.836545e-19,2.836545e-19
264999803,2265212,ANONYMOUS LOGON@C612,ANONYMOUS LOGON@C612,C612,C612,?,Network,LogOff,Success,27,3.578713e-17,3.578713e-17
264999928,2265212,C854$@DOM1,C854$@DOM1,C854,C1065,Kerberos,Network,LogOn,Success,27,6.800293e-02,6.800293e-02
264999992,2265213,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,27,1.883311e-17,1.883311e-17


,0,1,2,3,4,5,6,7,8,9,client_to_ip_pred,ip_to_service_pred
265000144,2265213,C854$@DOM1,C854$@DOM1,C854,C1065,?,?,TGS,Success,27,6.800293e-02,6.800293e-02
265000223,2265214,ANONYMOUS LOGON@C528,ANONYMOUS LOGON@C528,C528,C528,?,Network,LogOff,Success,27,2.598007e-17,2.598007e-17
265000226,2265214,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,27,2.836545e-19,2.836545e-19
265000228,2265214,ANONYMOUS LOGON@C612,ANONYMOUS LOGON@C612,C612,C612,?,Network,LogOff,Success,27,3.578713e-17,3.578713e-17
265000398,2265214,C854$@DOM1,C854$@DOM1,C854,C467,Kerberos,Network,LogOn,Success,27,6.800293e-02,6.800293e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
265999691,2271058,C2319$@DOM1,C2319$@DOM1,C467,C467,?,Network,LogOff,Success,27,8.804721e-02,8.804721e-02
265999835,2271059,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,27,1.883311e-17,1.883311e-17
265999839,2271059,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,27,2.836545e-19,2.836545e-19
265999842,2271059,ANONYMOUS LOGON@C612,ANONYMOUS LOGON@C612,C612,C612,?,Network,LogOff,Success,27,3.578713e-17,3.578713e-17


In [58]:
import re
invalid = 0
for chunk in pd.read_csv('../lm-vol/anomalies.csv', chunksize=100000):
#     display(chunk.head(500))
    client_to_ip_pred = []
    ip_to_service_pred = []
    for index, row in chunk.iterrows():
        try:
            client_to_ip = re.findall("\d+\.\d+", row[1])
            ip_to_service = re.findall("\d+\.\d+", row[2])
            client_to_ip_pred.append(float(client_to_ip[1]))
            ip_to_service_pred.append(float(ip_to_service[1]))
        except:
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            invalid+=1
#     print(client_to_ip_pred)
#     print(ip_to_service_pred)
    chunk['client_to_ip_pred'] = client_to_ip_pred
    chunk['ip_to_service_pred'] = ip_to_service_pred
    display(chunk)
    break

,Unnamed: 0,0,1,client_to_ip_pred,ip_to_service_pred
0,0,[[0.00280613 0.99719387]],[[0.00280613 0.99719387]],0.997194,0.997194
1,1,[[1.00000000e+00 1.86965761e-19]],[[1.00000000e+00 1.86965761e-19]],1.869658,1.869658
2,2,[[1.82675456e-05 9.99981732e-01]],[[1.82675456e-05 9.99981732e-01]],9.999817,9.999817
3,3,[[0.6802575 0.3197425]],[[0.00165419 0.99834581]],0.319742,0.998346
4,4,[[0.38826426 0.61173574]],[[0.38826426 0.61173574]],0.611736,0.611736
...,...,...,...,...,...
99995,99995,[[0.39758728 0.60241272]],[[0.39758728 0.60241272]],0.602413,0.602413
99996,99996,[[0.00124198 0.99875802]],[[0.00124198 0.99875802]],0.998758,0.998758
99997,99997,[[0.00923951 0.99076049]],[[0.00923951 0.99076049]],0.990760,0.990760
99998,99998,[[0.00319262 0.99680738]],[[0.00319262 0.99680738]],0.996807,0.996807


In [59]:
print(invalid)

150


In [60]:
print(len(client_to_ip_pred))

100000


In [61]:
anomalous_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)])
normal_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) > 0.1) & (chunk["client_to_ip_pred"].astype(float) > 0.1)])
print(anomalous_edges/(normal_edges+anomalous_edges))
print(anomalous_edges)
print(normal_edges)

0.015222834251377066
1520
98330


In [5]:
import pickle
import pandas as pd
import gensim
from gensim.models import KeyedVectors
import networkx as nx
experiment_names = ['2_18_500_epochs_v2']

def operator_hadamard(u, v):
        return u * v

for name in experiment_names:
    with open(f'../lm-vol/{name}_lr.pkl', 'rb') as file:
        clf = pickle.load(file)
    model = gensim.models.Word2Vec.load(f'../lm-vol/{name}_word2vec.model')
    wv = KeyedVectors.load(f'../lm-vol/{name}_word2vec.wordvectors', mmap='r')
    graph_structure = ('source', 'source_computer', 'destination')
    inconclusive = 0
    n = 0
    red = pd.read_csv('../lm-vol/redteam_complete.csv')
    client_to_ip_pred = []
    ip_to_service_pred = []
    for index, row in red.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    red['client_to_ip_pred'] = client_to_ip_pred
    red['ip_to_service_pred'] = ip_to_service_pred
    red_anomalies = red[(red["ip_to_service_pred"].astype(float) <= 0.1) | (red["client_to_ip_pred"].astype(float) <= 0.1)]
    red_invalids = red[(red["ip_to_service_pred"] == None) & (red["client_to_ip_pred"] == None)]
    display(red_anomalies)
    
    test_G = nx.read_graphml('../lm-vol/LANL_test.graphml')
    test = []
    # max = 100
    for index, edge in enumerate(test_G.edges):
    #     if index > max:
    #         break
        try:
            src = wv[edge[0]]
            dest = wv[edge[1]]
            src_to_dest = [operator_hadamard(src, dest)]
            prob = clf.predict_proba(src_to_dest)
            test.append(prob[0][1])
        except:
            test.append(None)
    anomalies = 0
    for prob in test:
        if prob is not None and prob <= 0.1:
            anomalies+=1
    print(anomalies)
    
#     graph_structure = (1, 3, 2)
#     client_to_ip_pred = []
#     ip_to_service_pred = []
#     inconclusive = 0
#     for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
#     #     display(chunk)
#         for index, row in chunk.iterrows():
#             try: 
#                 client_embedding = wv[(row[graph_structure[0]])]
#                 ip_embedding = wv[(row[graph_structure[1]])]
#                 service_embedding = wv[(row[graph_structure[2]])]
#                 client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
#                 ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
#                 client_to_ip = clf.predict_proba(client_to_ip_embedding)
#                 ip_to_service = clf.predict_proba(ip_to_service_embedding)
#                 client_to_ip_pred.append(client_to_ip[0][1])
#                 ip_to_service_pred.append(ip_to_service[0][1])
#             except: 
#                 client_to_ip_pred.append(None)
#                 ip_to_service_pred.append(None)
#                 inconclusive += 1
#         chunk['client_to_ip_pred'] = client_to_ip_pred
#         chunk['ip_to_service_pred'] = ip_to_service_pred
#         chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
#         chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
#         display(chunk_anomalies)
#         break
    break

,Unnamed: 0,time,source,source_computer,destination_computer,destination,authentication_type,logon_type,authentication_orientation,Success,client_to_ip_pred,ip_to_service_pred
50,50,725488,U1653@DOM1,C17693,C395,U1653@DOM1,NTLM,Network,LogOn,Success,0.000702,0.000702
51,51,725589,U1653@DOM1,C17693,C2669,U1653@DOM1,NTLM,Network,LogOn,Success,0.000702,0.000702
71,71,728800,U3905@DOM1,C17693,C9945,U3905@DOM1,NTLM,Network,LogOn,Success,0.036915,0.036915
72,72,729056,U995@DOM1,C17693,C1014,U995@DOM1,NTLM,Network,LogOn,Success,0.076066,0.076066
78,78,736582,U2837@DOM1,C17693,C89,U2837@DOM1,NTLM,Network,LogOn,Success,0.096589,0.096589
...,...,...,...,...,...,...,...,...,...,...,...,...
702,702,2305324,U12@DOM1,C17693,C366,U12@DOM1,NTLM,Network,LogOn,Success,0.047615,0.047615
703,703,2305554,U4448@DOM1,C17693,C1555,U4448@DOM1,NTLM,Network,LogOn,Success,0.058683,0.058683
705,705,2306980,U6572@DOM1,C17693,C90,U6572@DOM1,NTLM,Network,LogOn,Success,0.005804,0.005804
716,716,2461523,U3718@DOM1,C17693,C9692,U3718@DOM1,NTLM,Network,LogOn,Success,0.086488,0.086488


19231


In [8]:
import dask.dataframe as dd
df =  dd.read_csv('../lm-vol/LANL_test.csv', dtype='str', header= None, blocksize="1024MB")
display(df.head(100))

,0,1,2,3,4,5,6,7,8,9
0,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success,1
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,1
2,1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success,1
3,1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success,1
4,1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success,1
...,...,...,...,...,...,...,...,...,...,...
95,1,C660$@DOM1,SYSTEM@C660,C660,C660,Negotiate,Service,LogOn,Success,1
96,1,C674$@DOM1,SYSTEM@C674,C674,C674,Negotiate,Service,LogOn,Success,1
97,1,C688$@DOM1,C688$@DOM1,C688,C1065,Kerberos,Network,LogOn,Success,1
98,1,C693$@DOM1,C693$@DOM1,C528,C528,?,Network,LogOff,Success,1
